# Libraries


In [1]:
import numpy as np
import pandas as pd

# Parced data: feature + target

In [ ]:
ssc_v = "ssc_conv/{}_ssc.txt"

adj_m = "feature/adj/{}_adj.txt"
node_m = "feature/nodes/{}_node.txt"
vert_m = "feature/vertex/{}_vertex_count.txt"

count_file = "feature/counter_all.txt"

Open all the trash


In [ ]:
with open(count_file, 'r') as f:
    counter = f.read()
counter = counter.replace('\n', ' ')
counter = np.array([float(x) for x in counter.split()])
counter = counter.astype(np.int64)
len(counter)

In [ ]:
feature = []
feature_adj = []
feature_node = []
target = []
zaraza = []

size = 1000

for j in range (0, 3259): #len(counter)):
  i = counter[j]
  print(i)

  try:
    ssc = pd.read_csv(ssc_v.format(i))
    stress = ssc['stress [MPa]'].to_numpy()

    adj = np.loadtxt(adj_m.format(i))
    vert = np.loadtxt(vert_m.format(i))

    #node = np.loadtxt(node_m.format(i), delimiter=']')
    with open(node_m.format(i), 'r') as f:
        node = f.read()
    node = node.replace('[', '').replace(']', '').replace('\n', ' ')
    node = np.array([float(x) for x in node.split()]).reshape(-1, 3)


    #print(np.shape(adj))
    if len(adj)<=size:
      pads = size - len(adj)
      adj = np.pad(adj, ((0, pads), (0,pads)), mode = 'constant', constant_values = 0)
    else:
      adj = adj[:size, :size]

    if len(node)<=size:
      pads = size - len(node)
      node = np.pad(node, ((0, pads), (0, 0)), mode = 'constant', constant_values = 0)
      #print(np.shape(node))
    else:
      node = node[:size, :3]
      #print(np.shape(node))

    stress = np.concatenate((stress, np.zeros(size-len(stress))))
    stress = [float(item) for item in stress]

    feature_adj.append(adj)
    feature_node.append(node)
    feature.append(adj)
    target.append(stress)

  except Exception as e:
    print(i, 'zaraza')
    zaraza.append(i)

DNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(feature, target, test_size=0.05, random_state=42)
train_f, val_f, train_t, val_t = train_test_split(feature, target, test_size=0.1579, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', kernel_initializer='he_normal,  kernel_regularizer=l2(0.01)'))
model.add(Dense(72, activation='relu', kernel_initializer='he_normal,  kernel_regularizer=l2(0.01)'))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal,  kernel_regularizer=l2(0.01)'))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal,  kernel_regularizer=l2(0.01)'))
#model.add(Dense(24, activation='relu', kernel_initializer='he_normal,  kernel_regularizer=l2(0.01)'))
model.add(Dense(1, activation='linear',  kernel_initializer='he_normal,  kernel_regularizer=l2(0.01)'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(train_f, train_t,validation_data=(val_f, val_t), epochs=300, batch_size=50, verbose=1) 
model.summary()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
strain = []
for i_x in range(0, 7500, 5):
    i_x = i_x/100
    #print(i_x)
    strain.append(i_x)

s = 122

pred = model.predict(X_test[s])
plt.plot(strain, pred, 'X')
plt.plot(strain, y_test[s])
plt.xlabel('strain, [%]', fontsize = 15)
plt.ylabel('stress, [MPa]', fontsize = 15)
